In [ ]:
! pip install numpy #to install the NumPy package.

In [ ]:
! pip install pandas # to install pandas

In [ ]:
import pandas as pd #This line is importing the pandas library and giving it a short name pd.We use pd later in the code instead of writing pandas every time.Pandas helps in working with tables, spreadsheets, and data analysis.
import numpy as np # NumPy is super fast and helps when we want to do calculations on lists or large sets of numbers.

In [ ]:
news_data = pd.read_csv("C:\\Users\\anjup\\Downloads\\abcnews-date-text.csv")   #pd.read_csv(...) is a pandas function that loads the file into Python as a table.
news_data.shape  #.shape tells rows and columns

(1244184, 2)

In [ ]:
NUM_SAMPLES = 20000 # The number of sample to use

df = news_data.sample(NUM_SAMPLES, replace=False, random_state=42).reset_index(drop=True)

In [ ]:
! pip install gensim # used for topic modeling and word2vec

In [ ]:
! pip install nltk #Breaking text into words and sentences (tokenization)Removing stopwords (like "the", "is", "and")

In [ ]:
import nltk
nltk.download('punkt') #pre-trained model in nltk, helps in sentence and word tokenization


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anjup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\anjup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
import gensim
from gensim import corpora, models
import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# ----- Preprocessing -----
def preprocess(doc):
    tokens = nltk.word_tokenize(doc.lower())
    return [token for token in tokens if token.isalpha() and token not in stop_words]

texts = [preprocess(doc) for doc in df['headline_text'].dropna()]

# ----- Dictionary and BoW Corpus -----
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(text) for text in texts]

# ----- TF-IDF Transformation -----
tfidf_model = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf_model[bow_corpus]

# ----- LDA using TF-IDF corpus -----
lda_model = gensim.models.LdaModel(
    corpus=tfidf_corpus,
    id2word=dictionary,
    num_topics=6,
    random_state=42,
    passes=15,
    alpha='auto',
    per_word_topics=True
)

# Optional: Print topics
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")
    #This code takes news headlines, cleans and tokenizes them, turns them into vectors, then applies LDA to find 6 hidden topics.

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anjup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anjup\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Topic 0: 0.007*"new" + 0.006*"says" + 0.005*"nsw" + 0.005*"govt" + 0.005*"us" + 0.005*"council" + 0.005*"australia" + 0.005*"fire" + 0.004*"police" + 0.004*"plan"
Topic 1: 0.005*"market" + 0.005*"rise" + 0.004*"boost" + 0.004*"country" + 0.004*"ban" + 0.004*"opposition" + 0.004*"price" + 0.003*"tourism" + 0.003*"industry" + 0.003*"housing"
Topic 2: 0.006*"health" + 0.006*"death" + 0.005*"home" + 0.004*"indigenous" + 0.004*"drug" + 0.004*"nt" + 0.004*"farmers" + 0.004*"probe" + 0.004*"business" + 0.004*"missing"
Topic 3: 0.013*"man" + 0.007*"court" + 0.007*"police" + 0.006*"charged" + 0.006*"crash" + 0.005*"election" + 0.005*"car" + 0.005*"woman" + 0.005*"killed" + 0.005*"murder"
Topic 4: 0.004*"media" + 0.003*"highway" + 0.003*"charge" + 0.003*"shot" + 0.003*"arrest" + 0.003*"bush" + 0.003*"pacific" + 0.003*"floods" + 0.003*"delays" + 0.003*"long"
Topic 5: 0.010*"interview" + 0.006*"news" + 0.006*"rural" + 0.005*"national" + 0.005*"first" + 0.003*"afl" + 0.003*"violence" + 0.003*"minin

In [12]:
! pip install scikit-learn


In [13]:
from gensim.models import CoherenceModel
from sklearn.metrics import silhouette_score
import numpy as np

# ---------- Coherence Score ----------
coherence_model_lda = CoherenceModel(model=lda_model,
                                     texts=texts,
                                     dictionary=dictionary,
                                     coherence='c_v')
coherence_score_lda = coherence_model_lda.get_coherence()
print(" Coherence Score:", coherence_score_lda)

KeyboardInterrupt: 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
import numpy as np

# Step 1: Convert processed texts to TF-IDF vectors
processed_docs = [" ".join(text) for text in texts]  # Join tokens into strings
vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(processed_docs)

# Step 2: Assign LDA topic to each document
topic_assignments = []
for bow in [dictionary.doc2bow(text) for text in texts]:
    topic_probs = lda_model.get_document_topics(bow)
    top_topic = max(topic_probs, key=lambda x: x[1])[0]  # Most probable topic
    topic_assignments.append(top_topic)

labels = np.array(topic_assignments)

# Step 3: Compute Silhouette Score
if np.unique(labels).size > 1:
    sil_score = silhouette_score(tfidf_matrix, labels)
    print("📈 Silhouette Score (LDA + TF-IDF):", round(sil_score, 4))
else:
    print("⚠️ Not enough distinct topics for Silhouette Score.")

📈 Silhouette Score (LDA + TF-IDF): -0.1212


In [ ]:
from gensim import corpora, models

# Create dictionary and corpus
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(text) for text in texts]

# Train LDA model
lda_model = models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=6,
    passes=10,
    iterations=150,
    alpha='symmetric',
    eta='auto',
    random_state=42,
    per_word_topics=True
)

In [ ]:
# Customize this mapping based on your dataset
user_type_accounts = {
    0: 'Professional',
    1: 'soc.religion.christian',
    2: 'rec.sport.baseball',
    3: 'Mixed Type',
    4: 'Celebrity',
    5: 'talk.politics.misc'
}

In [ ]:
# Get dominant topic
topic_predictions = [lda_model.get_document_topics(doc, minimum_probability=0.1) for doc in bow_corpus]

# Predict user type based on dominant topicsdf
user_type_account_predictions = []

for topic_dist in topic_predictions:
    if topic_dist:
        dominant_topic = max(topic_dist, key=lambda x: x[1])[0]
    else:
        dominant_topic = -1  # Handle empty case
    user_type = user_type_accounts.get(dominant_topic, 'Unknown')
    user_type_account_predictions.append(user_type)

# Add to dataframe
df['predicted_user_type'] = user_type_account_predictions

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['topic_label'] = label_encoder.fit_transform(df['predicted_user_type'])

# Extract the label array for training
y = df['topic_label'].values

In [ ]:
import numpy as np

# Each document → topic distribution vector
corpus_bow = [dictionary.doc2bow(text) for text in texts]
topic_distributions = []

for doc_bow in corpus_bow:
    topic_dist = lda_model.get_document_topics(doc_bow, minimum_probability=0.0)
    topic_vector = [prob for _, prob in topic_dist]
    topic_distributions.append(topic_vector)

X = np.array(topic_distributions)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)
y_pred_log = log_reg.predict(X_test)

print("📊 Logistic Regression Accuracy:", round(accuracy_score(y_test, y_pred_log), 4))
print("\n📝 Classification Report (Logistic Regression):\n")
print(classification_report(y_test, y_pred_log))

📊 Logistic Regression Accuracy: 0.9673

📝 Classification Report (Logistic Regression):

              precision    recall  f1-score   support

           0       0.97      0.94      0.95       616
           1       0.95      0.99      0.97       672
           2       0.99      0.96      0.98       698
           3       0.98      0.97      0.98       685
           4       0.97      0.97      0.97       714
           5       0.93      0.98      0.96       615

    accuracy                           0.97      4000
   macro avg       0.97      0.97      0.97      4000
weighted avg       0.97      0.97      0.97      4000



In [ ]:
%history -f majorproj.py